In [1]:
####################################################################################
#
# Preprocessing_Patchfiles_and_Initial_Migrationfile.ipynb - script for pre-processing patchfiles and initial database migration file for the Joomla project.
# Copyright (C) 2023  Sravani Namburi
#
# Preprocessing_Patchfiles_and_Initial_Migrationfile.ipynb program is free software; you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation; either version 3 of the License, or
# (at your option) any later version.
# 
# Preprocessing_Patchfiles_and_Initial_Migrationfile.ipynb program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
# 
# You should have received a copy of the GNU General Public License along
# with Preprocessing_Patchfiles_and_Initial_Migrationfile.ipynb program; If not, see <https://www.gnu.org/licenses/>.
#
####################################################################################

In [2]:
import os
import re

### Pre-Processing Patch Files

In [3]:
#Extract Alter, create, rename, and drop statements from patchfiles
cwd = os.getcwd()
input_folder = os.path.join(cwd, 'Patchfiles')
output_folder = os.path.join(cwd, 'Patchfiles_Mod')


if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def remove_comments(sql):
    return re.sub(r'--.*?\n|/\*.*?\*/', '', sql, flags=re.DOTALL)

# Function to extract ALTER, CREATE, RENAME and DROP statements
def extract_statements(sql):
    statements = re.findall(r'(ALTER TABLE.*?;|CREATE TABLE.*?;|DROP TABLE.*?;|RENAME TABLE.*?;)', sql, flags=re.DOTALL)
    return statements

# Lines to skip, these tables are not present in the database.
lines_to_skip = [
    "ALTER TABLE `#__privacy_requests` MODIFY `requested_at` datetime NOT NULL;",
    "ALTER TABLE `#__privacy_requests` MODIFY `confirm_token_created_at` datetime NULL DEFAULT NULL;",
    "ALTER TABLE `#__privacy_consents` MODIFY `created` datetime NOT NULL;",
    "ALTER TABLE `#__action_logs` MODIFY `log_date` datetime NOT NULL;"
]


for filename in os.listdir(input_folder):
    if filename.endswith('.sql'):
        with open(os.path.join(input_folder, filename), 'r') as file:
            sql_content = file.read()

        sql_content = remove_comments(sql_content)
        
        statements = extract_statements(sql_content)

        output_filename = os.path.splitext(filename)[0] + '_mod.sql'
        output_filepath = os.path.join(output_folder, output_filename)
        with open(output_filepath, 'w') as file:
            for statement in statements:
                if statement.strip() not in lines_to_skip:
                    file.write(statement + '\n')

### Pre-Processing Initial Database Migration File

In [4]:
#Extract create table statements from initial database migration files

def extract_create_table_statements(sql_content):
    create_table_pattern = re.compile(r'CREATE TABLE .*?;', re.DOTALL)
    create_table_statements = create_table_pattern.findall(sql_content)
    return create_table_statements


def replace_date_and_default_values(sql_content):
    # Replace date values
    sql_content = sql_content.replace('0000-00-00', '2023-01-01')
    sql_content = sql_content.replace('0000-00-00 00:00:00', '2023-01-01 00:00:00')
    
    # Remove default values for specific data types
    sql_content = re.sub(r'(TEXT|MEDIUMTEXT|BLOB|GEOMETRY|JSON|text|mediumtext|blob|geometry|json) .*? DEFAULT \'\',', r'\1 NOT NULL,', sql_content)
    
    return sql_content


cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)

sql_file_path = os.path.join(parent_dir, 'joomla-4.0.0.sql')
with open(sql_file_path, 'r') as file:
    sql_file_content = file.read()

create_table_statements = extract_create_table_statements(sql_file_content)
modified_create_table_statements = [replace_date_and_default_values(statement) for statement in create_table_statements]

output_file_path = os.path.join(parent_dir, 'joomla-4.0.0_mod.sql')
with open(output_file_path, 'w') as file:
    file.write('\n'.join(modified_create_table_statements))